# Задания

Выполните задания, используя методы SQL Alchemy ORM

1. Создайте отображение базы данных студентов *Students_2021.sqlite*

In [3]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from datetime import datetime
engine = create_engine('sqlite:///Students_2021.sqlite')
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)
print(Base.classes.values())
city, exam_marks, student, university, subject, lecturer, subj_lect = Base.classes.values()

[<class 'sqlalchemy.ext.automap.city'>, <class 'sqlalchemy.ext.automap.exam_marks'>, <class 'sqlalchemy.ext.automap.student'>, <class 'sqlalchemy.ext.automap.university'>, <class 'sqlalchemy.ext.automap.subject'>, <class 'sqlalchemy.ext.automap.lecturer'>, <class 'sqlalchemy.ext.automap.subj_lect'>]


2. Напишите запрос, позволяющий получить из таблицы *exam_marks* значения столбца *mark* (экзаменационная оценка) для всех студентов, исключив из списка повторение одинаковых строк.

In [14]:
for item in session.query(exam_marks.mark).distinct(exam_marks.mark).all():
    print(item[0])

1
4
3
5
2
None


3. Напишите запрос для получения списка студентов без определенного места жительства.

In [30]:
for item in session.query(student.surname, student.name).filter(student.city.is_(None)).all():
    print(item[0], item[1])

Медведева Ирина
Афанасьева Оксана


4. Напишите запрос для получения списка студентов, проживающих в Воронеже и не получающих стипендию.

In [38]:
for item in session.query(student.surname, student.name).filter_by(city="Воронеж", stipend=0).all():
    print(item[0], item[1])

Павлов Андрей


5. Напишите запрос для получения списка университетов, расположенных в Москве и имеющих рейтинг меньший, чем у НГУ. Значение рейтинга НГУ получите с помощью отдельного запроса или подзапроса.

In [49]:
from sqlalchemy import and_

for item in session.query(university.univ_name).filter(and_(university.city=="Москва", university.rating<
                                                           session.query(university.rating).filter(university.univ_name=="НГУ").one()[0]
                                                           )).all():
    print(item[0])

ФинУ
МТУСИ


6. Напишите запрос, выполняющий вывод находящихся в таблице EXAM_MARKS номеров предметов обучения, экзамены по которым сдавались между 1 и 21 марта 2020 г.

In [56]:
import datetime

for item in session.query(exam_marks.subj_id).filter(exam_marks.exam_date.between(datetime.datetime.strptime('29-02-2020', "%d-%m-%Y").date(), datetime.datetime.strptime('22-03-2020', "%d-%m-%Y").date())).all():
    print(item[0])

73
12
10


7. Напишите запрос, который выполняет вывод названий предметов обучения, начинающихся на букву ‘И’.

In [58]:
for item in session.query(subject.subj_name).filter(subject.subj_name.like('И%')).all():
    print(item[0])

Информатика
История


8. Напишите запрос, выбирающий сведения о студентах, у которых имена начинаются на букву ‘И’ или ‘С’.

In [64]:
from sqlalchemy import or_

for item in session.query(student.surname, student.name).filter(or_(student.name.like('И%'), student.name.like('С%'))).all():
    print(item[0], item[1])

Иванов Иван
Дементьева Софья
Соколова Ия
Медведева Ирина
Кондрат Синицин
Мишина Светлана
Сорокина Ирина
Жданова Ирина
Блохина Ираида
Мельникова Светлана
Никитин Самуил
Назаров Икром
Захаров Софон
Колобов Иннокентий
Юдин Савва
Комаров Ипат
Симонов Семен


9. Напишите запрос для получения списка предметов обучения, названия которых состоят из более одного слова.

In [62]:
for item in session.query(subject.subj_name).filter(subject.subj_name.like('% %')).all():
    print(item[0])

Анализ данных


10. Напишите запрос для получения списка студентов, фамилии которых состоят из трех букв.

In [70]:
from sqlalchemy.sql.expression import func

for item in session.query(student.surname, student.name).filter(func.length(student.surname) == 3).all():
    print(item[0], item[1])

Шоу Бернар
Доу Джон


11. Составьте запрос для таблицы STUDENT таким образом, чтобы получить результат в следующем виде. Распечатайте первые 9 записей результата.
```
И. Иванов 	 1982-12-03
П. Петров 	 1980-12-01
В. Сидоров 	1979-06-07
...
```

In [135]:
from sqlalchemy import cast, String, Integer

for item in session.query(func.substr(cast(student.name, String(20)), 0, 2)+". "+
                          cast(student.surname, String(20))+"\t"+
                          func.substr(cast(student.birthday, String(20)), 0, 11)).limit(9).all():
    print(item[0])

И. Иванов	1982-12-03
П. Петров	1980-12-01
В. Сидоров	1979-06-07
Б. Кузнецов	1981-12-08
О. Зайцева	1981-05-01
П. Котов	2021-02-28
В. Белкин	1980-01-07
Е. Сергеева	1997-07-04
В. Кудряшова	2002-02-18


12. Напишите запрос для получения списка студентов, фамилии которых начинаются на ‘Ков’ или на ‘Куз’.

In [72]:
for item in session.query(student.surname, student.name).filter(or_(student.surname.like('Ков%'), student.surname.like('Куз%'))).all():
    print(item[0], item[1])

Кузнецов Борис
Ковалев Ефим


13. Напишите запрос для получения списка предметов, названия которых оканчиваются на ‘ия’.

In [73]:
for item in session.query(subject.subj_name).filter(subject.subj_name.like('%ия')).all():
    print(item[0])

История


14. Напишите запрос для выбора из таблицы EXAM_MARKS записей, для которых отсутствуют значения оценок (поле MARK).

In [77]:
for item in session.query(exam_marks.exam_id, exam_marks.student_id, exam_marks.subj_id, exam_marks.mark, exam_marks.exam_date).filter(exam_marks.mark.is_(None)).all():
    print(item)

(639, 55, 10, None, datetime.datetime(1999, 6, 22, 0, 0))


15. Составьте запрос, выводящий фамилии, имена студентов и величину получаемых ими стипендий, при этом значения стипендий должны быть увеличены в 100 раз.

In [137]:
for item in session.query(student.surname, student.name, cast(student.stipend, Integer)*100).all():
    for p in item:
        print(p, end=" ")
    print()

Иванов Иван 15000 
Петров Петр 20000 
Сидоров Вадим 15000 
Кузнецов Борис 0 
Зайцева Ольга 25000 
Котов Павел 15000 
Белкин Вадим 25000 
Сергеева Елизавета 15000 
Кудряшова Вера 10000 
Журавлева Вера 0 
Дементьева Софья 15000 
Рожкова Пелагея 0 
Соколова Ия 10000 
Семенова Вероника 0 
Медведева Ирина 10000 
Афанасьева Оксана 25000 
Сергеева Фёкла 10000 
Некрасова Вероника 25000 
Игнатьева Оксана 20000 
Казакова Ольга 25000 
Шубина Елена 0 
Миронова Анна 20000 
Кондрат Синицин 20000 
Пономарева Алина 25000 
Одинцова Лидия 10000 
Гуляева Рукижат 0 
Игнатова Зинаида 10000 
Мельникова Дарья 10000 
Мишина Светлана 15000 
Агафонов Василий 15000 
Афанасьева Анжелика 0 
Фролова Василиса 10000 
Лихачева Нина 15000 
Лапина Элеонора 15000 
Сорокина Ирина 20000 
Шашкова Галина 15000 
Казакова Мария 15000 
Жукова Фекла 15000 
Зимина Виктория 10000 
Жданова Ирина 20000 
Валерия Овчинникова 20000 
Костина Кира 0 
Блохина Ираида 25000 
Мельникова Светлана 25000 
Щербакова Агата 25000 
Васильева Оксана

16. Составьте запрос для таблицы UNIVERSITY таким образом, чтобы выходная таблица содержала всего один столбец в следующем виде: Код-10; ВГУ-г.ВОРОНЕЖ; Рейтинг=296.

In [122]:
for item in session.query("Код-"+cast(university.univ_id, String(20))+"; "+
                          cast(university.univ_name, String(20))+"-г."+
                          cast(university.city, String(20))+"; Рейтинг="+
                          cast(university.rating, String(20))+".").all():
    print(item[0])

Код-10; ВГУ-г.Воронеж; Рейтинг=296.
Код-11; НГУ-г.Новосибирск; Рейтинг=345.
Код-14; БГУ-г.Белгород; Рейтинг=326.
Код-15; ТГУ-г.Томск; Рейтинг=368.
Код-18; ВГМА-г.Воронеж; Рейтинг=327.
Код-22; МГУ-г.Москва; Рейтинг=400.
Код-32; РГУ-г.Ростов; Рейтинг=416.
Код-44; ФинУ-г.Москва; Рейтинг=330.
Код-45; МГТУ-г.Москва; Рейтинг=372.
Код-46; Политех-г.Санкт-Петербург; Рейтинг=300.
Код-47; КФУ-г.Казань; Рейтинг=330.
Код-48; УЛГУ-г.Ульяновск; Рейтинг=231.
Код-49; МТУСИ-г.Москва; Рейтинг=295.


17. Напишите запрос для подсчета количества студентов, сдававших экзамен по предмету обучения с идентификатором 10.

In [97]:
session.query(func.count(exam_marks.student_id)).filter(exam_marks.subj_id==10).all()[0][0]

10

18. Напишите запрос, который позволяет подсчитать в таблице EXAM_MARKS количество различных предметов обучения.

In [111]:
session.query(exam_marks.subj_id).distinct().count()

9

19. Напишите запрос, который для каждого студента выполняет выборку его идентификатора и минимальной из полученных им оценок.

In [108]:
for item in session.query(exam_marks.student_id, func.min(exam_marks.mark)).group_by(exam_marks.student_id).all():
    print(item[0], item[1])

1 1
6 4
10 5
12 5
15 5
32 4
55 5
62 4
64 4
65 1
71 3
73 1
76 2
77 1
79 1
82 5
83 1
85 1
88 5
90 4
91 5
92 5
97 1
99 5
101 2
103 2
105 3
110 1
116 1
117 4
123 2
126 1
128 3
149 2
203 3


20. Напишите запрос, который для каждого предмета обучения выводит наименование предмета и максимальное значение номера семестра, в котором этот предмет преподается.

In [112]:
for item in session.query(subject.subj_name, func.max(subject.semester)).group_by(subject.subj_name).all():
    print(item[0], item[1])

Анализ данных 1
Английский 3
Информатика 1
История 4
Математика 2
ОБЖ 2
Программирование 2
Физика 1
Физкультура 5


21. Напишите запрос, который для каждого конкретного дня сдачи экзамена выводит данные о количестве студентов, сдававших экзамен в этот день.

In [113]:
for item in session.query(exam_marks.exam_date, func.count(exam_marks.student_id)).group_by(exam_marks.exam_date).all():
    print(item[0], item[1])

1999-06-17 00:00:00 1
1999-06-22 00:00:00 1
2000-01-05 00:00:00 1
2000-01-18 00:00:00 1
2000-01-23 00:00:00 1
2006-01-12 00:00:00 1
2019-05-11 00:00:00 1
2019-06-08 00:00:00 1
2019-07-04 00:00:00 1
2019-07-06 00:00:00 1
2019-07-24 00:00:00 1
2019-08-14 00:00:00 1
2019-08-30 00:00:00 1
2019-09-02 00:00:00 1
2019-10-28 00:00:00 1
2019-12-17 00:00:00 1
2019-12-25 00:00:00 1
2019-12-31 00:00:00 1
2020-02-10 00:00:00 1
2020-02-22 00:00:00 1
2020-03-13 00:00:00 1
2020-03-20 00:00:00 1
2020-03-21 00:00:00 1
2020-04-06 00:00:00 1
2020-05-04 00:00:00 1
2020-05-10 00:00:00 1
2020-05-13 00:00:00 1
2020-07-26 00:00:00 1
2020-08-10 00:00:00 1
2020-08-18 00:00:00 1
2020-09-09 00:00:00 1
2020-09-21 00:00:00 1
2020-10-24 00:00:00 1
2020-12-03 00:00:00 1
2020-12-04 00:00:00 1
2020-12-23 00:00:00 1
2021-02-10 00:00:00 1
2021-03-01 00:00:00 1
2021-03-07 00:00:00 1
2021-03-14 00:00:00 1
2021-03-29 00:00:00 1


22. Напишите запрос, выдающий средний балл для каждого студента.

In [114]:
for item in session.query(exam_marks.student_id, func.avg(exam_marks.mark)).group_by(exam_marks.student_id).all():
    print(item[0], item[1])

1 1.0
6 4.0
10 5.0
12 5.0
15 5.0
32 4.0
55 5.0
62 4.0
64 4.0
65 1.0
71 3.0
73 1.0
76 2.0
77 1.0
79 1.0
82 5.0
83 1.0
85 2.5
88 5.0
90 4.0
91 5.0
92 5.0
97 1.0
99 5.0
101 2.0
103 2.0
105 3.0
110 1.0
116 1.0
117 4.0
123 2.0
126 1.0
128 4.0
149 2.0
203 3.0


23. Напишите запрос, выдающий средний балл для каждого экзамена.

In [115]:
for item in session.query(exam_marks.exam_id, func.avg(exam_marks.mark)).group_by(exam_marks.exam_id).all():
    print(item[0], item[1])

1 1.0
2 4.0
3 4.0
4 1.0
5 3.0
6 4.0
7 1.0
8 1.0
9 5.0
10 1.0
11 2.0
12 5.0
13 2.0
14 1.0
15 4.0
16 5.0
17 5.0
18 2.0
19 3.0
20 3.0
21 2.0
22 3.0
23 4.0
24 2.0
25 1.0
26 1.0
27 5.0
28 5.0
29 1.0
30 5.0
31 1.0
32 1.0
33 2.0
34 4.0
43 4.0
75 5.0
95 2.0
100 5.0
145 5.0
238 5.0
639 None


24. Напишите запрос, определяющий количество сдавших студентов для каждого экзамена.

In [138]:
for item in session.query(exam_marks.exam_id, func.count(exam_marks.student_id)).group_by(exam_marks.exam_id).all():
    print(item[0], item[1])

1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 1
28 1
29 1
30 1
31 1
32 1
33 1
34 1
43 1
75 1
95 1
100 1
145 1
238 1
639 1


25. Напишите запрос для определения количества предметов, преподаваемых на каждом курсе.

In [139]:
for item in session.query(cast(subject.semester, Integer)+1//2, func.count(subject.subj_id)).group_by(cast(subject.semester, Integer)+1//2).all():
    print(item[0], item[1])

1 3
2 3
3 1
4 1
5 1


26. Для каждого университета напишите запрос, выводящий суммарную стипендию.

In [141]:
for item in session.query(student.univ_id, func.sum(student.stipend)).group_by(student.univ_id).all():
    print(item[0], item[1])

10 3250
11 400
14 1700
15 650
18 1000
22 2180
32 1050
44 2220
45 650
46 1250
47 1450
48 1200


27. Для каждого семестра напишите запрос, выводящий общее количество часов, отводимое на изучение соответствующих предметов.

In [142]:
for item in session.query(subject.semester, func.sum(subject.hour)).group_by(subject.semester).all():
    print(item[0], item[1])

1 132
2 124
3 56
4 34
5 34
